In [1]:
from modules.preprocess import drop_rows
import pandas as pd  
from autogluon.tabular import TabularDataset, TabularPredictor

columns = [
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'educational num',
 'marital',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital gain',
 'capital loss',
 'hours per week',
 'country',
 'income'
 ]

#Load the dataset
train = pd.read_csv('data/adult.data', sep=",", names=columns)
test = pd.read_csv('data/adult.test', sep=",", names=columns)

#Drop missing
train = drop_rows(train, ['workclass', 'occupation', 'country'], drop_val =' ?')
test = drop_rows(test, ['workclass', 'occupation', 'country'], drop_val =' ?')

#preprocess
train['income'] = train['income'].str.strip(' .')
test['income'] = test['income'].str.strip(' .')

train_data = TabularDataset(train)
test_data = TabularDataset(test)

predictor = TabularPredictor(label='income',eval_metric = 'f1').fit(train_data, time_limit=2000,
                                                   presets='best_quality',auto_stack=True,)  # For a fair comparison with auto-sklearn

/home/claudio/Documents/DAS/Advanced_ML/Project/pyenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels/ag-20220824_174824/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 2000s
AutoGluon will save models to "AutogluonModels/ag-20220824_174824/"
AutoGluon Version:  0.5.2
Python Version:     3.8.8
Operating System:   Linux
Train Data Rows:    30162
Train Data Columns: 14
Label Column: income
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['<=50K', '>50K']
	If 'binary' is not the correct problem_type, please manually spec

In [2]:
predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L2   0.730697      15.233533  1056.352795                0.077604          31.240972            2       True         14
1           LightGBM_BAG_L1   0.725769       1.628893    12.380417                1.628893          12.380417            1       True          4
2            XGBoost_BAG_L1   0.725531       1.379618    22.819512                1.379618          22.819512            1       True         11
3      LightGBMLarge_BAG_L1   0.720661       6.560071    30.480087                6.560071          30.480087            1       True         13
4           CatBoost_BAG_L1   0.719717       0.491200   456.071168                0.491200         456.071168            1       True          7
5         LightGBMXT_BAG_L1   0.713135       1.845520    11.477874  

/home/claudio/Documents/DAS/Advanced_ML/Project/pyenv/lib/python3.8/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': 0.4193091968372867,
  'KNeighborsDist_BAG_L1': 0.4527216291922174,
  'LightGBMXT_BAG_L1': 0.7131348007480938,
  'LightGBM_BAG_L1': 0.7257693413862525,
  'R

# Test score best model (WeightedEnsemble_L2)

In [3]:
y_test = test['income']  # values to predict
test_nolab = test.drop(columns=['income'])  # delete label column to prove we're not cheating
y_pred = predictor.predict(test_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/claudio/Documents/DAS/Advanced_ML/Project/pyenv/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
Evaluation: f1 on test data: 0.715363804641891
Evaluations on test data:
{
    "f1": 0.715363804641891,
    "accuracy": 0.8672642762284196,
    "balanced_accuracy": 0.8037640369242482,
    "mcc": 0.6306852240073559,
    "precision": 0.7559434246163106,
    "recall": 0.6789189189189189
}


# Test score second best model (LightGBMXT_BAG_L1)

In [10]:
y_pred = predictor.predict(test_nolab, model='LightGBMXT_BAG_L1')
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: f1 on test data: 0.697498545666085
Evaluations on test data:
{
    "f1": 0.697498545666085,
    "accuracy": 0.8618857901726428,
    "balanced_accuracy": 0.7898110963075752,
    "mcc": 0.6116647664708779,
    "precision": 0.7550377833753149,
    "recall": 0.6481081081081081
}
